<a href="https://colab.research.google.com/github/kentokura/python_medical_images/blob/master/chapter3/chap3_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from google.colab.patches import cv2_imshow
from google.colab import drive
from IPython.display import Image, display_png

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3.10 画像と周波数(２次元フーリエ変換)


リスト 3.10.1

In [20]:
import plotly.express as px

pix = cv2.imread('/content/drive/My Drive/DATASET/dataset/ChestXray256_uint8.png',
                 cv2.IMREAD_GRAYSCALE)

pix_line = cv2.rectangle(np.copy(pix), (0, 128-1), (256, 128-1), (0, 255, 0), 3)
fig = px.imshow(pix_line, color_continuous_scale='gray') # plotlyで描画
fig.show()

c_row = pix[128-1, :]
data = {'intensity': c_row,
        'col': np.linspace(0, 255, 256)
        }
df = pd.DataFrame(data=data)
print(df.head())
fig = px.line(df, x='col', y='intensity', title='pixel intensities at row128')
fig.show()

   intensity  col
0        240  0.0
1        239  1.0
2        237  2.0
3        237  3.0
4        236  4.0
